<a href="https://colab.research.google.com/github/yaiza612/Big_data/blob/main/Consulting_proyect_predicting_dog_food_spoiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Yaiza Arnáiz Alcácer


Start Spark session


In [1]:
SPARK_VERSION = '3.4.0'
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ["SPARK_VERSION"] = '3.4.0'
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

Reading the file

In [2]:
df = spark.read.options(header='True', inferSchema='True').csv('/content/drive/MyDrive/Colab_Notebooks/dog_food.csv')
df.registerTempTable('data')

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [3]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)





Column 1.   Percentage of the preservative A in the mix

Column 2.   Percentage of the preservative B in the mix

Column 3.   Percentage of the preservative C in the mix

Column 4.   Percentage of the preservative D in the mix

Column 5.   Food batch spoiled






In [4]:
df.describe().toPandas()

,summary,A,B,C,D,Spoiled
0,count,490,490,490,490,490
1,mean,5.53469387755102,5.504081632653061,9.126530612244897,5.579591836734694,0.2857142857142857
2,stddev,2.9515204234399057,2.8537966089662063,2.0555451971054275,2.8548369309982857,0.45221563164613465
3,min,1,1,5.0,1,0.0
4,max,10,10,14.0,10,1.0


In [5]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [7]:
output = assembler.transform(df)

In [8]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
output.toPandas()

,A,B,C,D,Spoiled,features
0,4,2,12.0,3,1.0,"[4.0, 2.0, 12.0, 3.0]"
1,5,6,12.0,7,1.0,"[5.0, 6.0, 12.0, 7.0]"
2,6,2,13.0,6,1.0,"[6.0, 2.0, 13.0, 6.0]"
3,4,2,12.0,1,1.0,"[4.0, 2.0, 12.0, 1.0]"
4,4,2,12.0,3,1.0,"[4.0, 2.0, 12.0, 3.0]"
...,...,...,...,...,...,...
485,8,3,6.0,6,0.0,"[8.0, 3.0, 6.0, 6.0]"
486,6,4,9.0,10,0.0,"[6.0, 4.0, 9.0, 10.0]"
487,1,3,8.0,3,0.0,"[1.0, 3.0, 8.0, 3.0]"
488,6,6,8.0,3,0.0,"[6.0, 6.0, 8.0, 3.0]"


In [13]:
from pyspark.ml.classification import RandomForestClassifier

In [18]:
randomforest = RandomForestClassifier(featuresCol='features', labelCol='Spoiled')

In [19]:
final_data = output.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [20]:
randomforest_model = randomforest.fit(final_data)

In [21]:
randomforest_model.featureImportances

SparseVector(4, {0: 0.0139, 1: 0.0141, 2: 0.951, 3: 0.021})

The preservative C (index 2) is the responsible of the spoilage, because is the most important feature.



